# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import os
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import numpy as np

import warnings

warnings.simplefilter('ignore')

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/mafda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mafda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mafda/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
database_filepath = "../data/DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name, engine)

In [4]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [5]:
# Function to check which columns have unique values all equal to zero
def check_unique_zero_columns(df):
    """
    Checks for columns in the DataFrame that have unique values all equal to
    zero, and drops these columns from the DataFrame if found.

    Args:
        df (pd.DataFrame): The input DataFrame to check.

    Returns:
        pd.DataFrame: The DataFrame with zero-value columns dropped, if any were
        found.
    """
    zero_columns = []
    for column in df.columns:
        if df[column].nunique() == 1 and df[column].unique()[0] == 0:
            zero_columns.append(column)

    if len(zero_columns) > 0:
        print(f'{len(zero_columns)} columns have unique values all equal to zero')
        df = df.drop(zero_columns, axis=1)
        print(f'Drop {len(zero_columns)} columns')

    else:
        print(f'{len(zero_columns)} columns have unique values all equal to zero')
    return df

# Get the list of columns with unique values equal to zero
df = check_unique_zero_columns(df)
print(df.shape)

1 columns have unique values all equal to zero
Drop 1 columns
(26216, 39)


In [8]:
def random_messages(df, column, n):
    """
    Selects and prints a specified number of random messages from a given column
    in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing the messages. column
        (str): The name of the column to select messages from. 
        n (int): The number of random messages to select and print.

    Returns:
        None
    """
    random_messages = df[column].sample(n=n)
    print("Random messages:")
    for message in random_messages:
        print(message)

random_messages(df, 'message', 5)

Random messages:
I am hungry, I don't have food to eat, I don't have house I don't have clothes I count on you thank you so much 
Wind 16.0 mph NNE. Barometer 982.09 mb, gust 31.0, Temp 56.9 &amp;deg;F. Rain 0.00 in. Humidity 95% hurricane cam http://t.co/Sq2ekENuWind 16.0 mph NNE. Barometer 981.68 mb, gust 26.0, Temp 57.1 &amp;deg;F. Rain 0.00 in. Humidity 95% hurricane cam http://t.co/Sq2ekENu
Farmers have seen revenues hit by falling demand, the regional economic crisis, drought and mismanagement.
We're looking for more information concerning the earthquake
AFP correspondents witnessed poor turnout at polling stations across Bamako throughout the day, while residents contacted by telephone painted a similarly sparse picture in northern Mali.


In [9]:
def replace_urls(message, replacement='urlplaceholder'):
    """
    Replace URLs in a given message with a specified replacement string.

    Args:
        message (str): The input message containing URLs to be replaced.
        replacement (str, optional): The string to replace URLs with. Default is
        'urlplaceholder'.

    Returns:
        str: The message with URLs replaced by the specified replacement string.
    """
    url_pattern = r'(https?://\S+|www\.\S+)'
    return re.sub(url_pattern, replacement, message)

df['message'] = df['message'].apply(replace_urls)

In [10]:
# Extract X and y variables from the data for the modelling

X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

print(f'X shape {X.shape}, y shape {y.shape}')

X shape (26216,), y shape (26216, 35)


### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    words = nltk.word_tokenize(text)
    
    # remove stop words
    stopwords_ = nltk.corpus.stopwords.words("english")
    words = [word for word in words if word not in stopwords_]
    
    # extract root form of words
    words = [nltk.stem.WordNetLemmatizer().lemmatize(word, pos='v') for word in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

/Users/mafda/micromamba/envs/disaster_response/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
msg = ["I am hungry, I don't have food to eat, I don't have house I don't have clothes I count on you thank you so much"]
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'food' 'shelter' 'clothing'
 'direct_report']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args: actual: array. Array containing actual labels. predicted: array. Array
    containing predicted labels. col_names: list of strings. List containing
    names for each of the predicted fields.
       
    Returns: metrics_df: dataframe. Dataframe containing the accuracy,
    precision, recall and f1 score for a given set of actual and predicted
    labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average='weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average='weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df    

In [29]:
# Calculate evaluation metrics for test set
y_pred = pipeline.predict(X_test)
col_names = list(y.columns.values)

In [38]:
print(get_eval_metrics(np.array(y_test), y_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.808361   0.795991  0.808361  0.789256
request                 0.898535   0.893629  0.898535  0.887963
offer                   0.994812   0.989652  0.994812  0.992225
aid_related             0.795392   0.794001  0.795392  0.793920
medical_help            0.925236   0.908752  0.925236  0.897476
medical_products        0.951938   0.948186  0.951938  0.933377
search_and_rescue       0.974519   0.966012  0.974519  0.964357
security                0.981996   0.969479  0.981996  0.973671
military                0.968416   0.959948  0.968416  0.955793
water                   0.956820   0.956204  0.956820  0.948454
food                    0.950412   0.947739  0.950412  0.947358
shelter                 0.939579   0.935247  0.939579  0.929087
clothing                0.986878   0.987051  0.986878  0.981792
money                   0.980470   0.973266  0.980470  0.971527
missing_people          0.988099   0.976

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.